3. Predict all news data which is in folder normalized/ --- again just need date, news source, and then all the columns (we want column names of prediction as labels than code)

In [3]:
import pandas as pd
import numpy as np
import time

In [4]:
import re
import textwrap

def insert_chars_split_marker(text, cc=2500):
    # FIXME: text still has number
    text = re.sub('\d+', '', text)
    out = '|'.join(textwrap.wrap(text, cc))
    return out

def insert_words_split_marker(text, wc=500):
    text = re.sub('\d+', '', text)
    words = text.split()
    out = ''
    for i, w in enumerate(words):
        if i != 0 and i % wc == 0:
            out += '|' + w
        else:
            out += ' ' + w
    return out

In [5]:
from sklearn.externals import joblib
from sklearn.feature_extraction.text import TfidfTransformer

## Major Model

In [9]:
vect_name = '../models/vec_count_bills_23gram_new.joblib'
clf_name = '../models/major_bills_clf_liblinear_new.joblib'

In [10]:
vect = joblib.load(vect_name)
clf = joblib.load(clf_name)
len(vect.vocabulary_)

686467

In [13]:
total = 0
for df in pd.read_csv('../normalized/all-news.csv', iterator=True, chunksize=10000):
    if False:
        df['text'] = df['text'].apply(lambda c: insert_chars_split_marker(c, 2500))
        s = df['text'].str.split('|', expand=True).stack()
        i = s.index.get_level_values(0)
        new_df = df.loc[i].copy()
        new_df['chunk'] = s.index.get_level_values(1)
        new_df['text'] = s.values
        df = new_df.reset_index(drop=True)
    X = vect.transform(df.text.astype(str))
    tfidf = TfidfTransformer()
    X = tfidf.fit_transform(X)
    y_pred = clf.predict(X)
    df['topic'] = y_pred
    if total == 0:
        with open("../data/news_predict_major.csv", "w") as fp:
            df.to_csv(fp, index=False)
    else:
        with open("../data/news_predict_major.csv","a") as fp:
            df.to_csv(fp, header=False, index=False)
    total += 1
    print total

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
